# Time Series Convert Numeric and combine all
This Notebook is dividing the numerical values of one feature in high and low values. At the end all dimension (Vitalsigns Diagnosis etc.) are combined.

In [ ]:
import os
import time

import pyarrow.parquet as pq
import pandas as pd



# Vital Sign

In [ ]:
#defining the data paths
data_path = "Cohort/Time_Series/VitalSign_06.parquet"
target_path = "Cohort/Time_Series/VitalSign_06_quantile.parquet"
df = pq.read_table(data_path).to_pandas()

In [ ]:
#iterating over the Vital signs data frame and converting the numerical values
#df=df.head(100)
#df
unique_descriptions =list(df.context_procedure_code.unique())
#unique_descriptions
description_dict ={}

df["high_low"] = ""

description_dict = df.groupby(by='context_procedure_code')['numeric_value'].quantile([0.25,0.5,0.75]).to_dict()
description_dict
df.groupby(by="context_procedure_code")["numeric_value"].quantile([0.25, 0.5, 0.75]).head(100)


for key in description_dict:
    print(f'Key:{key} value{description_dict[key]}')
    #print(key[0])
    print('*'*100)
    high_cond=(df['context_procedure_code'] == key[0]) & (df['numeric_value']> description_dict[key])
    low_cond= (df['context_procedure_code'] == key[0]) & (df['numeric_value']<= description_dict[key])
    df['high_low'][high_cond] = key[0]+'_High'
    df['high_low'][low_cond] = key[0]+'_Low'
    print(df[df['context_procedure_code'] == key[0]]['high_low'].head(20))
    #break
print(df.head())
df
df.to_parquet(target_path)


# Lab Values

In [ ]:
#defining the data paths
data_path = "Cohort/Time_Series/LabValues_08.parquet"
target_path = "Cohort/Time_Series/LabValues_08_quantile.parquet"
df = pq.read_table(data_path).to_pandas()

In [ ]:
#df=df.head(100)
#df

In [ ]:
#iterating over the LabValues data frame and converting the numerical values
#df=df.head(10)
#df
unique_descriptions =list(df.test_name.unique())
#unique_descriptions
description_dict ={}

df["high_low"] = ""

description_dict = df.groupby(by='test_name')['numeric_value'].quantile([0.25,0.5,0.75]).to_dict()
description_dict
df.groupby(by="test_name")["numeric_value"].quantile([0.25, 0.5, 0.75]).head(10)


for key in description_dict:
    print(f'Key:{key} value{description_dict[key]}')
    #print(key[0])
    print('*'*100)
    high_cond=(df['test_name'] == key[0]) & (df['numeric_value']> description_dict[key])
    low_cond= (df['test_name'] == key[0]) & (df['numeric_value']<= description_dict[key])
    df['high_low'][high_cond] = key[0]+'_High'
    df['high_low'][low_cond] = key[0]+'_Low'
    print(df[df['test_name'] == key[0]]['high_low'].head(20))
    #break
print(df.head())
df
df.to_parquet(target_path)

In [ ]:
df

# Combine different Dimension 
MRN, Age in days, time detla high_low and other features as Event:


In [ ]:
#load lab values and drop NaN entries
df_lab_origin=pq.read_table('Cohort/Time_Series/LabValues_08_quantile.parquet').to_pandas()
df_lab=df_lab_origin[['medical_record_number','age_in_days','time_delta_in_days','high_low' ]]
df_lab=df_lab.rename(columns={"high_low": "event"})
df_lab=df_lab.dropna()
df_lab


In [ ]:
#load Vital Sign and drop NaN entries
df_vital_origin=pq.read_table('Cohort/Time_Series/VitalSign_06_quantile.parquet').to_pandas()
df_vital=df_vital_origin[['medical_record_number','age_in_days','time_delta_in_days','high_low' ]]
df_vital=df_vital.rename(columns={"high_low": "event"})
df_vital=df_lab.dropna()
df_vital

In [ ]:
#load diagnosiss and drop NaN entries
df_diagnosis_origin=pq.read_table('Cohort/Time_Series/Diagnosis_02.parquet').to_pandas()
df_diagnosis=df_diagnosis_origin[['medical_record_number','age_in_days','time_delta_in_days','context_diagnosis_code' ]]
df_diagnosis=df_diagnosis.rename(columns={"context_diagnosis_code": "event"})
df_diagnosis=df_diagnosis.dropna()
df_diagnosis

In [ ]:
#load Drug and drop NaN entries
df_drug_origin=pq.read_table('Cohort/Time_Series/Drug_02.parquet').to_pandas()
df_drug=df_drug_origin[['medical_record_number','age_in_days','time_delta_in_days','material_name' ]]
df_drug=df_drug.rename(columns={"material_name": "event"})
df_drug=df_drug.dropna()
df_drug

In [ ]:
#load procedures and drop NaN entries
df_procedures_origin=pq.read_table('Cohort/Time_Series/Procedures_06.parquet').to_pandas()
df_procedures=df_procedures_origin[['medical_record_number','age_in_days','time_delta_in_days','procedure_description' ]]
df_procedures=df_procedures.rename(columns={"procedure_description": "event"})
df_procedures=df_procedures.dropna()
df_procedures

In [ ]:
#combining all frames
frames=[df_lab,df_vital,df_diagnosis,df_drug,df_procedures]
all_time_series= pd.concat(frames)
all_time_series.to_parquet('Cohort/Time_Series/all_time_series_medium.parquet')

In [ ]:
all_time_series